### Definitions of the functions which are used to evaluate the best model for Sentinel of UAE and description of the nomenclature of different spectras which are used in the best models.

### continuum_removed function is used to find out the continuum removal of the smoothed spectra

In [3]:
def continuum_removal(points, show=False):
    x1, y1 = points.T
    augmented = np.concatenate([points, [(x1[0], np.min(y1)-1), (x1[-1], np.min(y1)-1)]], axis=0)
    hull = ConvexHull(augmented)
    continuum_points = points[np.sort([v for v in hull.vertices if v < len(points)])]
    continuum_function = interp1d(*continuum_points.T)
    yprime = continuum_function(x1) - y1
    #yprime = y1 / continuum_function(x1)

    if show:
        fig, axes = plt.subplots(2, 1, sharex=True)
        axes[0].plot(x1, y1, label='Data')
        axes[0].plot(*continuum_points.T, label='Continuum')
        axes[0].legend()
        axes[1].plot(x1, yprime, label='Data / Continuum')
        axes[1].legend()

    return yprime


def optimized_continuum_removed(spectra):
    cr_spec = spectra.copy()    
    row, col = spectra.shape
    x1 = np.arange(0, col, 1)
    
    def process_each_row(y1):
        points = np.c_[x1, y1]
        yprime = continuum_removal(points, show=False)
        return yprime

    cr_spec.values[:] = np.apply_along_axis(process_each_row, 1, cr_spec.values)
    
    return cr_spec

### fod function is used to find out the first order derivative of the various preprocessed spectras.


In [7]:
def fod (spectra):
    fo_spec = spectra.iloc[:,:].copy()
    (row,col) = fo_spec.shape
    
    for i in range(0,col):
        if i==col-1:
            fo_spec.iloc[:,i] = fo_spec.iloc[:,i-1]
        else:    
            fo_spec.iloc[:,i] = (spectra.iloc[:,i+1]- spectra.iloc[:,i])
        
    return(fo_spec.copy())